#Building AI Agent Assistant with OpenAI GPT-4o

Create a GPT-4o file search assistant that summarizes and explains arxiv papers about AGI.

The flow of this session is largely taken from the [OpenAI Assistants API documentation](https://platform.openai.com/docs/assistants/overview).

#### Notes

- OpenAI considers this much of this code experimental, so expect some changes in the coming months.

## Before you begin

- Make sure you have an OpenAI developer account.
- Your OpenAI developer account has credit on it.
- Define an environment variable named `OPENAI_API_KEY` containing the API key.

## Task 0: Setup

First we need to make sure that we are using the latest version of the OpenAI API package.

In [1]:
# Run this to install the latest version of the OpenAI package
!pip install openai==1.33.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


We need the `os`, `openai`, and `pandas` packages.

### Instructions

- Import the `os` and `openai` packages without an alias.
- Import the `pandas` package with its usual alias.

In [2]:
# Import the os package
import os

# Import the openai package
import openai

# Import the pandas package with an alias
import pandas as pd

We need to define an OpenAI client.

### Instructions

- Define an OpenAI client. Assign to `client`.

In [ ]:
# Define an OpenAI client. Assign to client.
client = openai.OpenAI()

## Task 1: Upload the Papers

So that GPT knows about the latest AGI research, we will provide it with some arxiv papers. There are 10 recent papers on AGI stored in the `papers` directory of this workbook.

_Click File -> Show workbook files to see a file browser._

_The papers were found by searching arxiv for "AGI", then eyballing recent papers for content on definitions of AGI or progress towards AGI._

The table below shows the filenames and the titles of the papers.

In [ ]:
# Run this
papers = pd.DataFrame({
    "filename": [
        "2405.10313v1.pdf",
        "2401.03428v1.pdf",
        "2401.09395v2.pdf",
        "2401.13142v3.pdf",
        "2403.02164v2.pdf",
        "2403.12107v1.pdf",
        "2404.10731v1.pdf",
        "2312.11562v5.pdf",
        "2311.02462v2.pdf",
        "2310.15274v1.pdf"
    ],
    "title": [
        "How Far Are We From AGI?",
        "EXPLORING LARGE LANGUAGE MODEL BASED INTELLIGENT AGENTS: DEFINITIONS, METHODS, AND PROSPECTS",
        "CAUGHT IN THE QUICKSAND OF REASONING, FAR FROM AGI SUMMIT: Evaluating LLMs’ Mathematical and Coding Competency through Ontology-guided Interventions",
        "Unsocial Intelligence: an Investigation of the Assumptions of AGI Discourse",
        "Cognition is All You Need The Next Layer of AI Above Large Language Models",
        "Scenarios for the Transition to AGI",
        "What is Meant by AGI? On the Definition of Artificial General Intelligence",
        "A Survey of Reasoning with Foundation Models",
        "Levels of AGI: Operationalizing Progress on the Path to AGI",
        "Systematic AI Approach for AGI: Addressing Alignment, Energy, and AGI Grand Challenges"
    ]
})
papers["filename"] = "papers/" + papers["filename"]
papers

,filename,title
0,papers/2405.10313v1.pdf,How Far Are We From AGI?
1,papers/2401.03428v1.pdf,EXPLORING LARGE LANGUAGE MODEL BASED INTELLIGE...
2,papers/2401.09395v2.pdf,"CAUGHT IN THE QUICKSAND OF REASONING, FAR FROM..."
3,papers/2401.13142v3.pdf,Unsocial Intelligence: an Investigation of the...
4,papers/2403.02164v2.pdf,Cognition is All You Need The Next Layer of AI...
5,papers/2403.12107v1.pdf,Scenarios for the Transition to AGI
6,papers/2404.10731v1.pdf,What is Meant by AGI? On the Definition of Art...
7,papers/2312.11562v5.pdf,A Survey of Reasoning with Foundation Models
8,papers/2311.02462v2.pdf,Levels of AGI: Operationalizing Progress on th...
9,papers/2310.15274v1.pdf,Systematic AI Approach for AGI: Addressing Ali...


To upload a file, you use `open()` to open it to get a file handle, then pass that handle to the client's `.files.create()` method. This returns details of the uploaded file, and the part we need to reuse is the file ID.


### Instructions

- Run this code to define a function to upload a file to the assistant.

In [ ]:
# Run this
def upload_file_for_assistant(file_path):
    uploaded_file = client.files.create(
        file=open(file_path, "rb"),
        purpose='assistants'
    )
    return uploaded_file.id

Now we apply the `upload_file_for_assistant()` function to each filename in the papers dataset to upload them.

### Instructions

- In `papers`, select the `filename` column, then apply `upload_file_for_assistant()`, then convert the result to a list. Assign to `uploaded_file_ids`.

In [ ]:
# In papers, select the filename column,
# then apply upload_file_for_assistant(),
# then convert the result to a list.
# Assign to uploaded_file_ids.
uploaded_file_ids = papers["filename"] \
    .apply(upload_file_for_assistant) \
    .to_list()

# See the result
uploaded_file_ids

['file-MJNrdzgWXsq5dkWRlwMFvl6X',
 'file-7Y3OWhfjFcT32CwjDEMYuSeg',
 'file-FWredybeCfIpGrbdLslnvcK8',
 'file-8AK41XPU9a7OKL9honKfLjwK',
 'file-dTxCJwyiiS1oQGoFKo5ksdtu',
 'file-yDPh4b3vMab9hu5FSXa3MsZQ',
 'file-zwhNIl8U3kX62o80ed539o5z',
 'file-TNwKe5iIVYlu6crDIMbKRZdj',
 'file-S66Y4sD0AGN0dl1K3pj81kDZ',
 'file-HZhROXHBtMSwbzofE1JY7nRQ']

### Check that this worked

View the files in your account at https://platform.openai.com/storage/files

## Task 2: Add the Files to a Vector Store

To access the documents and get sensible results, they need to be split up into small chunks and added to a vector database.

The assistants API lets you avoid worrying about the chunking stage, so you just need to specify the file IDs that you want to add to a vector database.

#### Notes

- You will get charged daily for having a vector database. By default, it will automatically be deleted after 7 days of not being used, but I suggest deleting it straight after this code-along if you don't want to be charged for a week.

### Instructions

- Create a vector store, associating the uploaded file IDs and naming it. (Suggested name: `arxiv_agi_papers`.) Assign to `vstore`.

<details>
  <summary>Code hints</summary>
  <p>

The code pattern for giving a vector store resource to a file search tool is as follows.
        
```py
vstore = client.beta.vector_stores.create(
    file_ids = file_ids,
    name = "vector store name"
)
```
        
  </p>
</details>   

In [ ]:
# Create a vector store, associating the uploaded file IDs and naming it.
vstore = client.beta.vector_stores.create(
    file_ids = uploaded_file_ids,
    name = "arxiv_agi_papers"
)

# See the result
vstore

VectorStore(id='vs_L0SSKEAAJzOMnnQN8hssgHwR', created_at=1718113596, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=10, total=10), last_active_at=1718113596, metadata={}, name='arxiv_agi_papers', object='vector_store', status='in_progress', usage_bytes=0, expires_after=None, expires_at=None)

### Check that this worked

View the vector stores in your account at https://platform.openai.com/storage/vector_stores

## Task 3: Create the Assistant

The assistant needs a prompt describing how it should behave. This consists of a few paragraphs of text that give GPT information about what its role is, what it should be talking about, and how to phrase the responses.

#### Pro tip

Just like any other writing, assistants prompt can be generated using ChatGPT (or any LLM). The prompt below was drafted by ChatGPT and had only minor human editing.

Here is the ChatGPT prompt I used to create the assistant prompt.

> I'm going to make a GPT assistant that explains the contents of journal articles about artificial general intelligence. The assistant, named 'Aggie', must be able to read arxiv papers in PDF form, and and explain the contents of those papers to an audience of data scientists. Please suggest a good instruction prompt for the AI assistant.

### Instructions

- Read the assistant prompt text to get a feel for what it is doing.
- Run the code to define the assistant prompt.

In [ ]:
# Run this
assistant_prompt = """
You are Aggie, a knowledgeable and articulate AI assistant specializing in artificial general intelligence (AGI). Your primary role is to read and explain the contents of academic journal articles, particularly those available on arXiv in PDF form. Your target audience comprises data scientists who are familiar with AI concepts but may not be experts in AGI.

When explaining the contents of the papers, follow these guidelines:

Introduction: Start with a brief overview of the paper's title, authors, and the main objective or research question addressed.

Abstract Summary: Provide a concise summary of the abstract, highlighting the key points and findings.

Key Sections and Findings: Break down the paper into its main sections (e.g., Introduction, Methods, Results, Discussion). For each section, provide a summary that includes:

The main points and arguments presented.
Any important methods or techniques used.
Key results and findings.
The significance and implications of these findings.
Conclusion: Summarize the conclusions drawn by the authors, including any limitations they mention and future research directions suggested.

Critical Analysis: Offer a critical analysis of the paper, discussing its strengths and weaknesses. Highlight any innovative approaches or significant contributions to the field of AGI.

Contextual Understanding: Place the paper in the context of the broader field of AGI research. Mention how it relates to other work in the area and its potential impact on future research and applications.

Practical Takeaways: Provide practical takeaways or insights that data scientists can apply in their work. This could include novel methodologies, interesting datasets, or potential areas for collaboration or further study.

Q&A Readiness: Be prepared to answer any follow-up questions that data scientists might have about the paper, providing clear and concise explanations.

Ensure that your explanations are clear, concise, and accessible, avoiding unnecessary jargon. Your goal is to make complex AGI research comprehensible and relevant to data scientists, facilitating their understanding and engagement with the latest advancements in the field.
"""

Now the assistant can be created. You simply give it a name, the prompt, the model to use (in this case GPT-4o), and specify which tools and resources it is allowed to use.

### Instructions

- Define the assistant. Assign to `aggie`.
    - Call it "Aggie" (or another memorable name).
    - Give it the `assistant_prompt`.
    - Set the model to use, `gpt-4o`.
    - Give it access to the file search tool.
    - Give it access to the vector store tool resource.

<details>
  <summary>Code hints</summary>
  <p>

The code pattern for creating a file search assistant is as follows.
        
```py
assistant = client.beta.assistants.create(
	name = "assistant name",
	instructions = prompt,
	model="gpt-4o",
	tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vstore.id]}}
)
```
        
  </p>
</details>   

In [ ]:
# Define the assistant. Assign to aggie.
aggie = client.beta.assistants.create(
	name = "Aggie",
	instructions = assistant_prompt,
	model="gpt-4o",
	tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vstore.id]}}
)

# See the result
aggie

Assistant(id='asst_W120u6wE1GdTXNt91Ok4KQVf', created_at=1718113597, description=None, instructions="\nYou are Aggie, a knowledgeable and articulate AI assistant specializing in artificial general intelligence (AGI). Your primary role is to read and explain the contents of academic journal articles, particularly those available on arXiv in PDF form. Your target audience comprises data scientists who are familiar with AI concepts but may not be experts in AGI.\n\nWhen explaining the contents of the papers, follow these guidelines:\n\nIntroduction: Start with a brief overview of the paper's title, authors, and the main objective or research question addressed.\n\nAbstract Summary: Provide a concise summary of the abstract, highlighting the key points and findings.\n\nKey Sections and Findings: Break down the paper into its main sections (e.g., Introduction, Methods, Results, Discussion). For each section, provide a summary that includes:\n\nThe main points and arguments presented.\nAny i

### Check that this worked

View the assistants in your account at https://platform.openai.com/playground/assistants

## Task 4: Create a Conversation Thread

Now you have an assistant, you can have a conversation. The first step in this is to create a thread object to contain the messages.

### Instructions

- Create a thread object. Assign to `conversation`.

<details>
  <summary>Code hints</summary>
  <p>

To create a conversation object, call `client.beta.threads.create()`.
        
  </p>
</details>  

In [ ]:
# Create a thread object. Assign to conversation.
conversation = client.beta.threads.create()

# See the result
conversation

Thread(id='thread_a35xewncnKCGMvyUamXArR8l', created_at=1718113598, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

Next you can add a message to the conversaation thread to ask a question.

### Instructions

- Add a user message to the conversation. Assign to `msg_what_is_agi`.
    - Give it the thread id.
    - Make it a user message.
    - Ask "What are the most common definitions of AGI?".

<details>
  <summary>Code hints</summary>
  <p>

The code pattern for creating a message is as follows.
        
```py
msg = client.beta.threads.messages.create(
    thread_id=conversation.id,
    role="user",
    content="your question"
)
```
        
  </p>
</details>   

In [ ]:
# Add a user message to the conversation. Assign to msg_what_is_agi.
msg_what_is_agi = client.beta.threads.messages.create(
    thread_id=conversation.id,
    role="user",
    content="What are the most common definitions of AGI?"
)

# See the result
msg_what_is_agi

Message(id='msg_0kQWBWyeoJvnbu3QsVRFAvk6', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What are the most common definitions of AGI?'), type='text')], created_at=1718113598, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_a35xewncnKCGMvyUamXArR8l')

## Task 5: Run the assistant

Running the assistant requires an event handler to make it print the responses. While it's fairly tricky code, you never need to change it. This code is taken verbatim from [the OpenAI assistants documentation](https://platform.openai.com/docs/assistants/overview).

### Instructions

- Run the code to define an event handler.

In [ ]:
# Run this
from typing_extensions import override
from openai import AssistantEventHandler

# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.

class EventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)

  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)


Finally, we are ready to run the assistant to get it to answer our question. The code is the same every time, so we can wrap it in a function.

Streaming responses mean that text is displayed a few words at a time, rather than waiting for the entirety of the text to be generated and printing all at once.

### Instructions

- Run the code to define the function.

In [ ]:
# Run this
def run_aggie():
    with client.beta.threads.runs.stream(
        thread_id=conversation.id,
        assistant_id=aggie.id,
        event_handler=EventHandler(),
    ) as stream:
        stream.until_done()

### Instructions

- Run the assistant.

In [ ]:
# Run the assistant
run_aggie()


assistant > file_search


assistant > The definitions of AGI (Artificial General Intelligence) are varied and often reflect different aspects of the concept. Here are the most common definitions extracted from the provided documents:

1. **Ability to Perform Human Cognitive Tasks:**
   - **Legg and Goertzel**: Popularized the term AGI by describing it as a machine capable of performing cognitive tasks typically done by humans, focusing on non-physical tasks【4:0†source】.

2. **Learning and Adaptability:**
   - **Shanahan**: AGI is an artificial intelligence that is not specialized in specific tasks but can learn to perform a broad range of tasks as humans do【4:0†source】.
   - **Bowen Xu**: Defines AGI as systems that adapt to open environments with limited resources, emphasizing the importance of learning ability in intelligence【4:3†source】【4:4†source】.

3. **Economic Measure:**
   - **OpenAI**: Defines AGI as highly autonomous systems that outperform humans at most economically valuab

## Task 6: Add Another Message and Run it Again

Since we've gone to the trouble of creating an assistant, we might as well ask more questions.

### Instructions

- Create another user message, adding it to the conversation. This time, ask "How close are we to developing AGI?". Assign to `msg_how_close_is_agi`.

In [ ]:
# Create another user message, adding it to the conversation. Assign to msg_how_close_is_agi.
msg_how_close_is_agi = client.beta.threads.messages.create(
    thread_id=conversation.id,
    role="user",
    content="How close are we to developing AGI?"
)

# See the result
msg_how_close_is_agi

Message(id='msg_XHt5lYFmth5dpVLAN6o8qlIu', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='How close are we to developing AGI?'), type='text')], created_at=1718113617, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_a35xewncnKCGMvyUamXArR8l')

### Instructions

- Run the assistant again.

In [ ]:
# Run the assistant
run_aggie()


assistant > file_search


assistant > The recent advancements in AI have brought us significantly closer to AGI (Artificial General Intelligence), but many researchers believe that we are still several years or even decades away from achieving full AGI. Here are the key points drawn from the literature on this topic:

1. **Current Capabilities and Advances**:
    - AI systems have shown remarkable progress in specific domains such as natural language processing (NLP), computer vision, and reinforcement learning. Large language models (LLMs) like GPT-4 have demonstrated multi-task solving capabilities that surpass human performance in some areas【4:0†source】【4:1†source】【4:13†source】.
    - AI has achieved superhuman performance in narrowly defined tasks, such as complex board games like Go and various benchmarks used in NLP and image recognition【4:13†source】.

2. **Levels of AGI**:
    - Researchers propose different levels of AGI development. Currently, we are at "Level 1" where AI sys

## Want to learn more?

# ***`Try working the same with GEMINI API KEY`***